In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tfs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D,LeakyReLU
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2, random
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
"""
Reference Credits:
-Dataset details and source https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones
-Code reference & step by step explanation source: https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/


Some notes:
9 features, each in a file:
    -The “Inertial Signals” directory contains 9 files.
    --Gravitational acceleration data files for x, y and z axes: total_acc_x_train.txt, total_acc_y_train.txt, total_acc_z_train.txt.
    --Body acceleration data files for x, y and z axes: body_acc_x_train.txt, body_acc_y_train.txt, body_acc_z_train.txt.
    --Body gyroscope data files for x, y and z axes: body_gyro_x_train.txt, body_gyro_y_train.txt, body_gyro_z_train.txt.
    
    Each of the 9 features are listed below
    Total Acceleration x
    Total Acceleration y
    Total Acceleration z
    Body Acceleration x
    Body Acceleration y
    Body Acceleration z
    Body Gyroscope x
    Body Gyroscope y
    Body Gyroscope z
6 classes or type-of-activity to get trained on and predict:
    1 WALKING
    2 WALKING_UPSTAIRS
    3 WALKING_DOWNSTAIRS
    4 SITTING
    5 STANDING
    6 LAYING

The data has been split into windows of 128 time steps, with a 50% overlap.
trainX = (7352, 128, 9) 7352 samples or windows, each 128 timesteps or readings, represented with 9 features where each feature data is in a separate file
testX(2947, 128, 9)

trainY (7352, 1), activity type performed (one of the six activities)
testY  (2947, 1)
"""

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from  keras.utils import np_utils

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	x_train, y_train = load_dataset_group('train', prefix + '/content/drive/MyDrive/data/HAR/')
	print(x_train.shape, y_train.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + '/content/drive/MyDrive/data/HAR/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	y_train = y_train - 1
	testy = testy - 1
	# one hot encode y
	# y_train = np_utils.to_categorical(y_train)
	# testy = np_utils.to_categorical(testy)
	print(x_train.shape, y_train.shape, testX.shape, testy.shape)
	return x_train, y_train, testX, testy

#load the data
'''   .
trainX = (7352, 128, 9) 7352 samples or windows, each 128 timesteps or readings, represented with 9 features where each feature data is in a separate file
testX(2947, 128, 9)
trainY (7352, 1), activity type performed (one of the six activities)
testY  (2947, 1)
'''
x_train, y_train, testX, testy = load_dataset()
n_timesteps, n_features = x_train.shape[1], x_train.shape[2]
n_outputs=6
x_train=x_train.reshape(x_train.shape[0],n_timesteps, n_features)
testX=testX.reshape(testX.shape[0],n_timesteps, n_features)


y_train=y_train.reshape(-1)
testy=testy.reshape(-1)


(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 1) (2947, 128, 9) (2947, 1)


In [ ]:
from sklearn.utils import shuffle
x_train,y_train = shuffle(x_train, y_train, random_state=42)
testX,testy=shuffle(testX,testy, random_state=42)
ix=0
trainX=list()
trainy=list()
for ind11 in range(40):
  trainX.append(x_train[ix:ix+600])
  trainy.append(y_train[ix:ix+600])
  # trainX.append(x_train[ix+183:ix+366+183])
  # trainy.append(y_train[ix+183:ix+366+183])
  ix=ix+170
  print(ix)

170
340
510
680
850
1020
1190
1360
1530
1700
1870
2040
2210
2380
2550
2720
2890
3060
3230
3400
3570
3740
3910
4080
4250
4420
4590
4760
4930
5100
5270
5440
5610
5780
5950
6120
6290
6460
6630
6800


In [ ]:
trainX=np.array(trainX)
trainy=np.array(trainy)
# Y=np.reshape(Y, (40,360))

print(trainX.shape,trainy.shape)
# print(trainy[0])
X=trainX
Y=trainy
x_test=testX
y_test=testy
print(y_test.shape)

(40, 600, 128, 9) (40, 600)
(2947,)


In [ ]:
from tensorflow.keras.layers import BatchNormalization
class MODEL:
    @staticmethod
    def build(nl):
        model = Sequential()
        model.add(Conv1D(128,3,strides=2, padding="same",input_shape=(n_timesteps, n_features)) )
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling1D(pool_size=2, strides=1, padding="same"))
        model.add(Dropout(0.25))

        # model.add(Conv2D(128/nl, (3, 3), strides=(2, 2), padding="same") )
        model.add(Conv1D(64/nl, 3, strides=2, padding="same") )
        
        model.add(Conv1D(128/nl, 3, strides=2, padding="same") )
        model.add(Conv1D(256/nl, 3, strides=2, padding="same") )
        model.add(Conv1D(512/nl, 3, strides=2, padding="same") )
        model.add(Flatten())
        # model.add(Dense(256/nl,activation='relu')) #add if acc <90
        # model.add(Dense(32/nl,activation='relu'))
        model.add(Dense(6,activation='softmax'))#for output layer
        return model

In [ ]:
from sklearn.metrics import accuracy_score
def test_model(X_test, Y_test,  model, comm_round):
    # cce = tf.keras.losses.SparseCategoricalCrossentropy()
    # #logits = model.predict(X_test, batch_size=100)
    # logits = model.predict(X_test)
    # loss = cce(Y_test, logits)
    # acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    # print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    model.compile(  optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
    loss,acc=model.evaluate(x_test,y_test)
    # acc,loss=model.evaluate(x_test,y_test)
    return acc, loss
def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
      
    return avg_grad
# print(global_model.get_weights())
def test_modelstudent(X_test, Y_test,  gmodel, comm_round):
    gmodel.compile(  optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
    loss,acc=gmodel.evaluate(x_test,y_test)
    # acc,loss=gmodel.evaluate(x_test,y_test)

    return acc, loss

In [ ]:
import numpy as np
import csv

path = 'survey.csv'
with open(path, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader)
    data = np.array(list(reader)).astype(float)
print(data.shape)
# print(data)
data=data[:40]
data=np.delete(data, 0, 1)
print(data.shape)
data[:, [2, 0]] = data[:, [0, 2]]
print(data)

(100, 4)
(40, 3)
[[ 2.1  10.88  8.  ]
 [ 0.9   4.1   3.  ]
 [ 2.2   1.13  6.  ]
 [ 2.7  11.45  3.  ]
 [ 1.6   8.9   3.  ]
 [ 3.1   2.    4.  ]
 [ 1.4   8.7   1.  ]
 [ 1.4  60.    3.  ]
 [ 1.    8.89  3.  ]
 [ 2.8  34.5   8.  ]
 [ 1.7  12.54  6.  ]
 [ 1.9   1.2   6.  ]
 [ 1.6  28.41  6.  ]
 [ 2.5  21.9   3.  ]
 [ 1.8  25.99  6.  ]
 [ 2.7  19.43  4.  ]
 [ 1.6  20.98  3.  ]
 [ 1.9  30.    3.  ]
 [ 2.1  12.    2.  ]
 [ 2.6  10.    6.  ]
 [ 1.5  40.    1.  ]
 [ 2.3  11.4   6.  ]
 [ 2.3  25.    6.  ]
 [ 2.2  30.    4.  ]
 [ 3.1   9.62  6.  ]
 [ 1.6  23.27  6.  ]
 [ 1.2  49.79  6.  ]
 [ 2.7  37.65  6.  ]
 [ 2.8  15.71  6.  ]
 [ 0.9   3.    6.  ]
 [ 1.8  18.04  6.  ]
 [ 2.8  44.13  6.  ]
 [ 0.8   6.5   6.  ]
 [ 2.4  60.21  6.  ]
 [ 1.9  61.3   8.  ]
 [ 1.2  19.    6.  ]
 [ 1.3  32.05  6.  ]
 [ 2.3   6.52  6.  ]
 [ 2.3  38.8   6.  ]
 [ 0.9  32.    6.  ]]


In [ ]:
maxminarr=[]
sz=len(data[0])
for i in range(sz):
  tmp=[np.max(data,axis=0)[i],np.min(data,axis=0)[i]]
  maxminarr.append(tmp)
print(maxminarr)

[[3.1, 0.8], [61.3, 1.13], [8.0, 1.0]]


In [ ]:
V=data
normV=[]
for i in range(len(V)):
  tmpar=[]
  for j in range(len(V[i])):
    tmp=(V[i][j]-maxminarr[j][1])/(maxminarr[j][0]-maxminarr[j][1])
    # print(tmp)
    # V[i][j]=tmp
    tmpar.append(tmp)
  # print(tmpar.shape)
  normV.append(tmpar)
    
    

# print(V)
print(normV)

[[0.5652173913043479, 0.1620408841615423, 1.0], [0.04347826086956521, 0.04936014625228519, 0.2857142857142857], [0.6086956521739132, 0.0, 0.7142857142857143], [0.8260869565217392, 0.17151404354329403, 0.2857142857142857], [0.3478260869565218, 0.1291341199933522, 0.2857142857142857], [1.0, 0.014459032740568393, 0.42857142857142855], [0.26086956521739124, 0.1258102044208077, 0.0], [0.26086956521739124, 0.9783945487784611, 0.2857142857142857], [0.08695652173913042, 0.12896792421472497, 0.2857142857142857], [0.8695652173913043, 0.5545953132790428, 1.0], [0.391304347826087, 0.1896293834136613, 0.7142857142857143], [0.47826086956521735, 0.0011633704503905612, 0.7142857142857143], [0.3478260869565218, 0.45338208409506403, 0.7142857142857143], [0.7391304347826088, 0.3451886322087419, 0.2857142857142857], [0.4347826086956522, 0.4131627056672761, 0.7142857142857143], [0.8260869565217392, 0.30413827488781786, 0.42857142857142855], [0.3478260869565218, 0.32989862057503744, 0.2857142857142857], [0.

In [ ]:
pov=[]
lam=[0.4,0.4,0.2]#[1.0/3,1.0/3,1.0/3]
for i in range(len(normV)):
  tmp=0
  for j in range(len(normV[i])):
    tmp+=lam[j]*normV[i][j]
  pov.append(tmp)
print(pov)

[0.4909033101863561, 0.0942782199915973, 0.38633540372670816, 0.45618325716887054, 0.24792693992280673, 0.4914978988105131, 0.1546719078552796, 0.5528485027411981, 0.1435126355243993, 0.7696642122681387, 0.37523063535304224, 0.33462683886338607, 0.46334041127777725, 0.4908704839393974, 0.48203526860231427, 0.5378043782781086, 0.3282327401554809, 0.44037009012766115, 0.3269203096402843, 0.5148668833749511, 0.38014032704438877, 0.4719999339345974, 0.5624104375078066, 0.521115431742568, 0.5992972292789478, 0.4291705591920202, 0.5359058237684738, 0.7160707188844858, 0.587608607909061, 0.17267989144628523, 0.4291850109988573, 0.776539969052488, 0.17855599610627035, 0.8138718764742133, 0.791304347826087, 0.33121910281118616, 0.4353646036024226, 0.4395585179465634, 0.6541505073100337, 0.3654669946538638]


In [ ]:
import math
def euclidean_distance(V):
  dist=[]
  for i in range(len(V)):
    tmp=[]
    for j in range(len(V)):
      disc=0
      lambd=[0.4,0.4,0.2]#[1.0/3,1.0/3,1.0/3]
      for ind in range(len(V[i])):
        disc+=math.sqrt(lambd[ind]*(V[i][ind]-V[j][ind])**2)
      tmp.append(disc)
    dist.append(tmp)
  return dist

In [ ]:
d=euclidean_distance(normV)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
DIAMETER_METHODS = ['mean_cluster', 'farthest']
CLUSTER_DISTANCE_METHODS = ['nearest', 'farthest']


def inter_cluster_distances(labels, distances, method='nearest'):
    """Calculates the distances between the two nearest points of each cluster.
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: `nearest` for the distances between the two nearest points in each cluster, or `farthest`
    """
    if method not in CLUSTER_DISTANCE_METHODS:
        raise ValueError(
            'method must be one of {}'.format(CLUSTER_DISTANCE_METHODS))

    if method == 'nearest':
        return __cluster_distances_by_points(labels, distances)
    elif method == 'farthest':
        return __cluster_distances_by_points(labels, distances, farthest=True)


def __cluster_distances_by_points(labels, distances, farthest=False):
    n_unique_labels = len(np.unique(labels))
    cluster_distances = np.full((n_unique_labels, n_unique_labels),
                                float('inf') if not farthest else 0)

    np.fill_diagonal(cluster_distances, 0)

    for i in np.arange(0, len(labels) - 1):
        for ii in np.arange(i, len(labels)):
            if labels[i] != labels[ii] and (
                (not farthest and
                 distances[i][ii] < cluster_distances[labels[i], labels[ii]])
                    or
                (farthest and
                 distances[i][ii] > cluster_distances[labels[i], labels[ii]])):
                cluster_distances[labels[i], labels[ii]] = cluster_distances[
                    labels[ii], labels[i]] = distances[i][ii]
    return cluster_distances


def diameter(labels, distances, method='farthest'):
    """Calculates cluster diameters
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: either `mean_cluster` for the mean distance between all elements in each cluster, or `farthest` for the distance between the two points furthest from each other
    """
    if method not in DIAMETER_METHODS:
        raise ValueError('method must be one of {}'.format(DIAMETER_METHODS))

    n_clusters = len(np.unique(labels))
    diameters = np.zeros(n_clusters)

    if method == 'mean_cluster':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii]:
                    diameters[labels[i]] += distances[i][ii]

        for i in range(len(diameters)):
            diameters[i] /= sum(labels == i)

    elif method == 'farthest':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii] and distances[i][ii] > diameters[
                        labels[i]]:
                    diameters[labels[i]] = distances[i][ii]
    return diameters


def dunn(labels, distances, diameter_method='farthest',
         cdist_method='nearest'):
    """
    Dunn index for cluster validation (larger is better).
    
    .. math:: D = \\min_{i = 1 \\ldots n_c; j = i + 1\ldots n_c} \\left\\lbrace \\frac{d \\left( c_i,c_j \\right)}{\\max_{k = 1 \\ldots n_c} \\left(diam \\left(c_k \\right) \\right)} \\right\\rbrace
    
    where :math:`d(c_i,c_j)` represents the distance between
    clusters :math:`c_i` and :math:`c_j`, and :math:`diam(c_k)` is the diameter of cluster :math:`c_k`.
    Inter-cluster distance can be defined in many ways, such as the distance between cluster centroids or between their closest elements. Cluster diameter can be defined as the mean distance between all elements in the cluster, between all elements to the cluster centroid, or as the distance between the two furthest elements.
    The higher the value of the resulting Dunn index, the better the clustering
    result is considered, since higher values indicate that clusters are
    compact (small :math:`diam(c_k)`) and far apart (large :math:`d \\left( c_i,c_j \\right)`).
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param diameter_method: see :py:function:`diameter` `method` parameter
    :param cdist_method: see :py:function:`diameter` `method` parameter
    
    .. [Kovacs2005] Kovács, F., Legány, C., & Babos, A. (2005). Cluster validity measurement techniques. 6th International Symposium of Hungarian Researchers on Computational Intelligence.
    """

    labels = LabelEncoder().fit(labels).transform(labels)

    ic_distances = inter_cluster_distances(labels, distances, cdist_method)
    min_distance = min(ic_distances[ic_distances.nonzero()])
    max_diameter = max(diameter(labels, distances, diameter_method))
    # print(min_distance,max_diameter)

    return min_distance / max_diameter

In [ ]:
dunnarray=[]
dunnvalues=[]
rng=int(math.sqrt(len(data)))
print(rng)
for ncluster in range(2,rng+1):
      kmeans = KMeans(n_clusters=ncluster)
      # c = data[1]

      k = kmeans.fit_predict(normV)
      # d = euclidean_distances(x)
      print("No of Clusters(K)= ",ncluster)
      # print(k)

      for diameter_method in DIAMETER_METHODS:
          for cdist_method in CLUSTER_DISTANCE_METHODS:
              # dund = dunn(c, d, diameter_method, cdist_method)
              
              if diameter_method=="farthest" and cdist_method=="nearest":
                dunk = dunn(k, d, diameter_method, cdist_method)
                tmp=[ncluster,dunk]
                dunnarray.append(tmp)
                dunnvalues.append(dunk)


                print(diameter_method, cdist_method, dunk)

6
No of Clusters(K)=  2
farthest nearest 0.06653752129046049
No of Clusters(K)=  3
farthest nearest 0.0859314010630813
No of Clusters(K)=  4
farthest nearest 0.19035956869003984
No of Clusters(K)=  5
farthest nearest 0.12420275823622348
No of Clusters(K)=  6
farthest nearest 0.14012546089492353


In [ ]:
bestnoofcluster=dunnvalues.index(max(dunnvalues))+2
print(bestnoofcluster)
d=[]
for i in range(len(dunnvalues)):
  d.append([dunnvalues[i],i+2])
print(d)
d.sort()
d.reverse()
print(d)
aa=[b for a,b in d[:1]]
print(aa)
best3kclusters=aa

if(best3kclusters[0]>2):
    best3kclusters.append(best3kclusters[0]-1)
    best3kclusters.append(best3kclusters[0]-2)
print(best3kclusters)

4
[[0.06653752129046049, 2], [0.0859314010630813, 3], [0.19035956869003984, 4], [0.12420275823622348, 5], [0.14012546089492353, 6]]
[[0.19035956869003984, 4], [0.14012546089492353, 6], [0.12420275823622348, 5], [0.0859314010630813, 3], [0.06653752129046049, 2]]
[4]
[4, 3, 2]


In [ ]:
# Ef=20
# u=0.7
# L=1.5
# B=1
# Ew=0.08
# beta=max(8*L/u,Ef)
# qf=0.05
# Rf=(1/Ef)*(((L/(2*u*u*qf))*(4*B+u*u*beta*Ew*Ew))+1-beta)
# print(Rf)
# import math
# Rf=math.ceil(Rf)
# print(Rf)

In [ ]:
nc=bestnoofcluster

In [ ]:
lerr=np.random.uniform(low=0.02, high=0.05, size=(40))
# lerr=np.array([0.04675046,0.02234626,0.03282212,0.034653,0.027742,0.0304253,
#  0.03549977, 0.03090621, 0.04324615, 0.03075343, 0.04978067, 0.03210354,
#  0.02494483, 0.03837972, 0.04388003, 0.02834624, 0.03452465, 0.02933506,
#  0.02621393, 0.04510132, 0.0218163, 0.04455994, 0.04109578, 0.03545095,
#  0.04047112, 0.03039103, 0.03500365, 0.04162522, 0.02930526, 0.04583842,
#  0.02061416, 0.03754902, 0.04033513, 0.03562403, 0.02130191, 0.04754293,
#  0.03351984, 0.04711428, 0.02425473, 0.02373612])
print(lerr)
def herr(jj,C,i):
  sumx=0
  lenx=0
  maxx=0
  minn=10
  # return len(C[jj])
  for ix in range(len(C[jj])):
    sumx+=lerr[C[jj][ix]]
    lenx+=1
    maxx=max(maxx,lerr[C[jj][ix]])
    minn=min(minn,lerr[C[jj][ix]])
  sumx+=lerr[i]
  lenx+=1
  avgerr=sumx/lenx
  tmp=max(abs(maxx-avgerr),abs(avgerr-minn))
  return avgerr,tmp


[0.03069942 0.04795441 0.04573769 0.04727083 0.02888882 0.02724254
 0.03676593 0.04443896 0.03622262 0.02224869 0.02644186 0.03026514
 0.03690504 0.02161341 0.04579484 0.03308182 0.04479164 0.03703423
 0.02911798 0.04510812 0.02559712 0.04526577 0.03303454 0.02860894
 0.04228355 0.02739375 0.04763538 0.02355647 0.03903153 0.03394206
 0.02486551 0.04673203 0.04296839 0.04820669 0.0338767  0.02126551
 0.0301952  0.04506277 0.04187602 0.0490344 ]


In [ ]:

# f.write(str(lerr))
# f.write(str(nc))

In [ ]:
for nc in best3kclusters:
  print("\n","Number of Clusters=",nc,"\n")
  f = open("MyFileHARwithoutKD.txt", "a+") 
  f.write("\n Number of Clusters=%d \n" %nc)
  f1=open("HAR GLobalwithoutKD.txt","a+")
  f1.close()
  kmeans = KMeans(n_clusters=nc)
  k = kmeans.fit_predict(normV)
  print("Client with their cluster number",k,"\n")
  f.write("Client with their cluster number %s \n"%k)
  AccomodateM=[0 for i in range(nc)]
  su=0
  lent=0
  for i in range(nc):
    su=0
    lent=0
    for j in range(len(k)):
      if(k[j]==i):
          su+=pov[j]
          lent=lent+1
    AccomodateM[i]=su/lent
  AccomodateM.sort()
  AccomodateM.reverse()  
  AccomodateM.append(0)
  print("Cluster value for which it can accomodate any client with value around this",AccomodateM,"\n")
  f.write("Cluster value for which it can accomodate any client with value around this %s \n"%AccomodateM)




  tmp=200

  # Rf=[(tmp*10) for i in range(nc)]
  R=[]
  for i in range(nc):
    if(i==0):
      R.append(tmp)
      tmp=200
    else:
      R.append(tmp)
    # tmp=math.ceil(tmp/1.2)
  E=[]
  tmp=10
  epochlow=1
  epochmax=5
  tmp=epochmax
  divfactor=(epochmax-epochlow)/nc
  for i in range(nc):
    E.append(int(tmp))
    tmp=(tmp-divfactor)
  
  print("Global Comm Rounds",R)
  f.write("Global Comm Rounds %s \n"%R)
  print("Local Epochs",E,"\n")
  f.write("Local Epochs %s \n"%E)
  localepoch=[0 for i in range(len(pov))]
  addndata=[[] for i in range(nc)]





  qo=np.random.uniform(low=0.02, high=0.05, size=nc)
  addtmp=(0.05-0.02)/nc
  qo=[]
  tmp=0.02
  for i in range(nc):
    qo.append(round(tmp,3))
    tmp=tmp+addtmp
  tmp=0.05

  delta=[]
  tmp=0.03
  for i in range(nc):
    delta.append(tmp)
    tmp=tmp*1.00001
  print("Qo",qo)
  print("delta",delta)
  f.write("Qo %s \n" %qo)
  f.write("Delta %s \n" %delta)
  errf=[]
  tmp=0.1
  for i in range(nc):
    errf.append(tmp)
    tmp=round(tmp*1.2,3)
  print("Err",errf)
  theta=[]
  tmp=0.03
  for i in range(nc):
    theta.append(tmp)
    tmp=round(tmp*1.2,3)
  print("Theta",theta,"\n")
  f.write("Theta %s \n" %theta)


  C=[[] for i in range(nc)]
  accmthreshold=0.1
  for i in range((len(pov))):
    flag=0
    for j in range(len(C)):
      if(flag==1):
        continue
      if(i==1 and j==1):
        print("hello")
      if len(C[j])==0:
        if( (pov[i]-AccomodateM[j]>=0) or (abs(pov[i]-AccomodateM[j])-abs(pov[i]-AccomodateM[j+1])<=0) ):
          # print("(pov[i]-AccomodateM[j]>0)",(pov[i]-AccomodateM[j]))
          # print("(pov[i]-AccomodateM[j])",i,(pov[i]-AccomodateM[j]))
          # print("pov[i]-AccomodateM[j+1]",i,pov[i]-AccomodateM[j+1])
          #calculate qof
          clustrloss=lerr[i]
          if(clustrloss<=delta[j]):
            qo[j]=clustrloss
            errf[j]=0
            C[j].append(i)
            flag=1
            localepoch[i]=E[j]

        else:
          #increase data in this client
          dwkcwdkkc=0
          qofRE=E[j]/R[j]
          rat=qo[j]/qofRE
          newlocalEpoch=E[j]-1
          
          newqofRE=newlocalEpoch/R[j]
          

          clustrloss=rat*newqofRE
          # print("Clustrloss",clustrloss,"0")
          
          if(clustrloss<=delta[j] and clustrloss>=0.02):
            qo[j]=clustrloss
            errf[j]=0
            C[j].append(i)
            localepoch[i]=E[j]-1
            addndata[j].append(i)
            flag=1
            # print("Delta","Client",i,"Cluster",j,clustrloss,delta[j],0,localepoch[i])
          
          # print("LOss,",qo[j],qofRE)
          #reduce ti and ni s.t. pi run Mf
      else:

        if( (pov[i]-AccomodateM[j]>=0) or (abs(pov[i]-AccomodateM[j])-abs(pov[i]-AccomodateM[j+1])<=0) or j==len(C)-1):
          #calculate qof
          clustrlossavg,hetroloss=herr(j,C,i)
          #  print('i',i,'j',j,tmp)
          if(clustrlossavg<=delta[j] and hetroloss<=theta[j]):
            errf[j]=hetroloss
            qo[j]=clustrlossavg
            C[j].append(i)
            flag=1
            localepoch[i]=E[j]
          # else:
          #   print('i',i,'j',j,tmp,'theta',theta[j])
          #   print("CLusterlossavg",clustrlossavg,"Delta[j]",delta[j],"Hetroloss",hetroloss,"Theta[j]",theta[j])


        else:
          dwkcwdkkc=0
          qofRE=E[j]/R[j]
          rat=qo[j]/qofRE
          newlocalEpoch=E[j]-1
          newqofRE=newlocalEpoch/R[j]

          clustrloss=rat*newqofRE
          tmpvar=lerr[i]
          lerr[i]=clustrloss
          

          # clustrloss=lerr[i]
          # print("Clustrloss",clustrloss,"1")
          
          clustrlossavg,hetroloss=herr(j,C,i)
          # deltanewj=rat*clustrlossavg
          
          if(clustrlossavg<=delta[j] and hetroloss<=theta[j] and clustrloss>=0.02):

            errf[j]=hetroloss
            qo[j]=clustrlossavg
            C[j].append(i)
            flag=1
            localepoch[i]=E[j]-1
            addndata[j].append(i)
            # print("Delta","Client",i,"Cluster",j,clustrlossavg ,delta[j],1,localepoch[i])
          
          
          else:
            lerr[i]=tmpvar

          #reduce ti and ni s.t. pi run Mf

      if(flag==0 and j== len(C)-1):
        if(len(C[j])==0):
          clustrloss=lerr[i]
          qo[j]=clustrloss
          errf[j]=0
          C[j].append(i)
          flag=1
          localepoch[i]=E[j]
        else:
          clustrlossavg,hetroloss=herr(j,C,i)
          #  print('i',i,'j',j,tmp)
          errf[j]=hetroloss
          qo[j]=clustrlossavg
          C[j].append(i)
          localepoch[i]=E[j]
          flag=1


      
  for tmx in range(nc):
    C[tmx]=np.array(C[tmx])
  print("Clustering Array",np.array(C,dtype=object))
  f.write("Clustering Array %s \n" %(np.array(C,dtype=object)))
  print("ERRf of each cluster",errf,"\n")
  f.write("ERRf of each cluster %s \n"%errf)
  print("Qo of each cluster",qo,"\n")


  #Model Training
  print("Model Training","\n")
  f.write("Model Training\n")
  flag=1
  # for empclstr in range(nc):
  #   if(len(C[empclstr])==0):
  #     # flag=0
  #     f.write("Cluster no. %d is empty" %empclstr)
  #     break
  print("Original Cluster count",nc)
  emptcnt=0
  # if(len(C[nc-1])==0):
    #   emptcnt=emptcnt+1
  for abd in range(0,nc):
    # print("abd",abd)
    if(len(C[abd])==0):
      cx=abd
      while(len(C[cx])==0 and cx!=nc-1):
        
        cx=cx+1
      C[abd],C[cx]=C[cx],C[abd]
      addndata[abd],addndata[cx]=addndata[cx],addndata[abd]
      print("CX",cx)
      print("ABD",abd)
     # emptcnt=emptcnt+1
  for abd in range(0,nc):
    if(len(C[abd])==0):
      emptcnt=emptcnt+1

  print("C",C)
  
  print("emptcnt",emptcnt) 
  nc=nc-emptcnt
  emptcnt=0
  print("NC",nc)

  #shifting clients
  for abd in range(nc-1,0,-1): 
    if(len(C[abd])<=4 and len(C[abd])>0):

      for val in C[abd]:
        C[abd-1]=np.append(C[abd-1],int(val))
        # print("Val",val,"abd",abd)
      for val in addndata[abd]:
        addndata[abd-1]=np.append(addndata[abd-1],int(val))
      C[abd]=[]
      addndata[abd]=[]
      emptcnt=emptcnt+1
    # print("ABD here",abd)
  print("C11",C)
  var1=1
  if(len(C[0])<=4 and len(C[0])>0):
    while(1):
      if(len(C[0])+len(C[var1])>=5):
        break
      var1+=1
    for val in C[0]:
        C[var1]=np.append(C[var1],int(val))
    for val in addndata[0]:
      addndata[var1]=np.append(addndata[var1],int(val))
    C[0]=[]
    addndata[0]=[]
    emptcnt=emptcnt+1
    
  print("C1",C)
  
  # print("emptcnt1",emptcnt) 
  # print("NC1",nc)
  for abd in range(0,nc):
    # print("abd",abd)
    if(len(C[abd])==0):
      cx=abd
      while(len(C[cx])==0 and cx!=nc-1):
        
        cx=cx+1
      C[abd],C[cx]=C[cx],C[abd]
      # print("CX",cx)
      # print("ABD",abd)
    # print("NC1",nc)
  for abd in range(0,nc):
    # print("abd",abd)
    if(len(addndata[abd])==0):
      cx=abd
      while(len(addndata[cx])==0 and cx!=nc-1):
        
        cx=cx+1
      addndata[abd],addndata[cx]=addndata[cx],addndata[abd]
      # print("CX",cx)
      # print("ABD",abd)
  print("C2",C)
  nc=nc-emptcnt
  print("emptcnt2",emptcnt) 
  print("NC2",nc)
  print("Updated ADDitional Data",addndata)
  f.write("Final Clustering Array %s \n" %(np.array(C,dtype=object)))
    
      





  f.close()
  X=[[] for i in range(40)]
  Y=[[] for i in range(40)]
  for clstr in range(nc):
      data_size=len(x_train)
      ix=0
      for ind11 in range(len(C[clstr])):
        datarang=int(data_size/len(C[clstr]))
        X[C[clstr][ind11]]=x_train[ix:ix+2*datarang]
        Y[C[clstr][ind11]]=y_train[ix:ix+2*datarang]
        # trainX.append(x_train[ix+183:ix+366+183])
        # trainy.append(y_train[ix+183:ix+366+183])
        if(len(C[clstr])-1>0):
          step=(data_size-2*datarang)/(len(C[clstr])-1)

        print("step",step)
        ix=ix+int(step)
        print("ix",ix)
  X=np.array(X)
  Y=np.array(Y)

  print(X.shape,Y.shape)
  for i in X:
    print("inside X",i.shape)
  if(flag==1):
    for clstr in range(nc):
      # clstr=4
      f = open("MyFileHARwithoutKD.txt", "a+") 
      f1=open("HAR GLobalwithoutKD.txt","a+")
      print("CLUSTER NO.",clstr,"\n")
      f.write("CLUSTER NO.%d \n" %clstr)
      f1.write("CLuster NO. %d \n" %clstr)
      f1.close()
      if(clstr==0):
        
        global1= MODEL()
        global_model = global1.build(1)
        for comm_round in range(R[clstr]):
          f1=open("HAR GLobalwithoutKD.txt","a+")
          global_weights = global_model.get_weights()
          scaled_local_weight_list = list()
          index=list({0,1,2,3,4})
          #random.shuffle(index)
          print(index)
          tsf=0
          for clnt in range(len(C[clstr])):
            print("CLIENT NO: ",C[clstr][clnt])

            local = MODEL()
            local_model=local.build(1)
            
            
            local_model.compile(optimizer=keras.optimizers.Adam(),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=[keras.metrics.SparseCategoricalAccuracy()],
        )
            local_model.set_weights(global_weights)
            
            #trainy[ind]=np.array(trainy[ind]).reshape(-1,1)
            if(C[clstr][clnt] in addndata[clstr]):
              print("Clnt",C[clstr][clnt],"epoch",localepoch[C[clstr][clnt]])
              history=local_model.fit(X[C[clstr][clnt]],Y[C[clstr][clnt]], epochs=localepoch[C[clstr][clnt]])#,validation_data=(x_test,y_test))
              # print("Accuracy: ",history.history["accuracy"][4])
              local_model.evaluate(x_test, y_test)
              scaling_factor=len(X[C[clstr][clnt]])/(len(C[clstr])*len(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100])+100*len(addndata[clstr])) #1/no.ofclients
              tsf+=scaling_factor
            else:
              # print("Clnt",clnt,"epoch",localepoch[clnt])
              history=local_model.fit(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100],Y[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100], epochs=localepoch[C[clstr][clnt]])#,validation_data=(x_test,y_test))
              # print("Accuracy: ",history.history["accuracy"][4])
              local_model.evaluate(x_test, y_test)
              scaling_factor=len(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100])/(len(C[clstr])*len(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100])+100*len(addndata[clstr])) #1/no.ofclients
              tsf+=scaling_factor

            
            print("No of clients in cluster",len(C[clstr]))
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            scaled_local_weight_list.append(scaled_weights)
            K.clear_session()
          print("Total Scaling Factor",tsf)
          average_weights = sum_scaled_weights(scaled_local_weight_list)
          #global_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
          global_model.set_weights(average_weights)
          #val_acc,val_loss=global_model.evaluate(x_train,y_train)
          #print(val_loss,val_acc)
          global_acc, global_loss = test_model(x_test,y_test, global_model, comm_round)
          print("COMM_ROUNND",comm_round," GLOBAL MASTER MODEL ACCURACY",global_acc,global_loss)
          f.write("Communication Round: %d GLOBAL MASTER MODEL ACCURACY: %f LOSS: %f \n" %(comm_round ,global_acc ,global_loss))
          f1.write("%f \t %f \n" %(global_acc,global_loss))
          f1.close()
      else:
        
        studentm=MODEL.build(2**clstr)
        student_model=studentm#Distiller(student=studentm, teacher=global_model)
        for comm_round in range(R[clstr]):
          f1=open("HAR GLobalwithoutKD.txt","a+")
          global_weights = student_model.get_weights()
          scaled_local_weight_list = list()
          index=list({0,1,2,3,4})
          #random.shuffle(index)
          print(index)
          tsf=0
          for clnt in range(len(C[clstr])):
            print("CLIENT NO: ",C[clstr][clnt])
            local_model1 = MODEL.build(2**clstr)#Distiller(student=studentm, teacher=global_model)
            
            local_model1.compile(optimizer=keras.optimizers.Adam(),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=[keras.metrics.SparseCategoricalAccuracy()],
        )

            local_model1.set_weights(global_weights)
            #trainy[ind]=np.array(trainy[ind]).reshape(-1,1)

            if(C[clstr][clnt] in addndata[clstr]):
              print("Clnt",C[clstr][clnt],"epoch",localepoch[C[clstr][clnt]])
              history=local_model1.fit(X[C[clstr][clnt]],Y[C[clstr][clnt]], epochs=localepoch[C[clstr][clnt]])#,validation_data=(x_test,y_test))
              # print("Accuracy: ",history.history["accuracy"][4])
              local_model1.evaluate(x_test, y_test)
              scaling_factor=len(X[C[clstr][clnt]])/(len(C[clstr])*len(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100])+100*len(addndata[clstr])) #1/no.ofclients
              tsf+=scaling_factor
            else:
              # print("Clnt",clnt,"epoch",localepoch[clnt])
              history=local_model1.fit(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100],Y[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100], epochs=localepoch[C[clstr][clnt]])#,validation_data=(x_test,y_test))
              # print("Accuracy: ",history.history["accuracy"][4])
              local_model1.evaluate(x_test, y_test)
              scaling_factor=len(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100])/(len(C[clstr])*len(X[C[clstr][clnt]][:len(X[C[clstr][clnt]])-100])+100*len(addndata[clstr])) #1/no.ofclients
              tsf+=scaling_factor


            scaled_weights = scale_model_weights(local_model1.get_weights(), scaling_factor)
            scaled_local_weight_list.append(scaled_weights)
            K.clear_session()
          print("Total Scaling Factor",tsf)  
          average_weights = sum_scaled_weights(scaled_local_weight_list)
          #global_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
          student_model.set_weights(average_weights)
          #val_acc,val_loss=global_model.evaluate(x_train,y_train)
          #print(val_loss,val_acc)
          global_acc1, global_loss1 = test_modelstudent(x_test,y_test, student_model, comm_round)
          print("COMM_ROUNND",comm_round,"GLOBAL SLAVE MODEL ACCURACY",global_acc1,global_loss1)
          f.write("Communication Round: %f GLOBAL SLAVE MODEL ACCURACY: %f LOSS:%f \n" %(comm_round,global_acc1 ,global_loss1))
          f1.write("%f \t %f \n" %(global_acc1,global_loss1))
          f1.close()
      f.close()














  







 Number of Clusters= 4 

Client with their cluster number [0 1 0 0 1 0 1 1 1 2 3 0 3 1 3 0 1 1 1 0 1 0 0 1 0 3 3 2 0 3 3 2 3 2 2 3 3
 0 2 3] 

Cluster value for which it can accomodate any client with value around this [0.7536002719692411, 0.48942439141548916, 0.38165039071022844, 0.33462614442591465, 0] 

Global Comm Rounds [1, 1, 1, 1]
Local Epochs [5, 4, 3, 2] 

Qo [0.02, 0.028, 0.035, 0.043]
delta [0.03, 0.0300003, 0.030000600003000002, 0.030000900009000034]
Err [0.1, 0.12, 0.144, 0.173]
Theta [0.03, 0.036, 0.043, 0.052] 

hello
Clustering Array [array([ 9, 27, 34, 35, 36, 38, 39]) array([1])
 array([ 4,  5, 10, 12, 13, 15, 20, 22, 23, 25, 28, 30])
 array([ 0,  2,  3,  6,  7,  8, 11, 14, 16, 17, 18, 19, 21, 24, 26, 29, 31,
        32, 33, 37])                                                       ]
ERRf of each cluster [0.015486768900811701, 0, 0.009639454429053326, 0.012149228031046965] 

Qo of each cluster [0.026389253375892208, 0.021, 0.029392072780728112, 0.04126720305194588] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:342: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:343: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[0, 1, 2, 3, 4]
CLIENT NO:  9
Epoch 1/5
55/55 [==============================] - 1s 12ms/step - loss: 0.7585 - sparse_categorical_accuracy: 0.6732
Epoch 2/5
55/55 [==============================] - 1s 12ms/step - loss: 0.4708 - sparse_categorical_accuracy: 0.8078
Epoch 3/5
55/55 [==============================] - 1s 12ms/step - loss: 0.4236 - sparse_categorical_accuracy: 0.8234
Epoch 4/5
55/55 [==============================] - 1s 12ms/step - loss: 0.2891 - sparse_categorical_accuracy: 0.8797
Epoch 5/5
93/93 [==============================] - 1s 4ms/step - loss: 0.5981 - sparse_categorical_accuracy: 0.8415
No of clients in cluster 8
CLIENT NO:  27
Epoch 1/5
55/55 [==============================] - 1s 13ms/step - loss: 0.7729 - sparse_categorical_accuracy: 0.6738
Epoch 2/5
55/55 [==============================] - 1s 13ms/step - loss: 0.4065 - sparse_categorical_accuracy: 0.8274
Epoch 3/5
55/55 [==============================] - 1s 13ms/step - loss: 0.3620 - sparse_categorical_accuracy: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:342: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:343: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Epoch 1/5
19/19 [==============================] - 1s 12ms/step - loss: 0.9572 - sparse_categorical_accuracy: 0.6150
Epoch 2/5
19/19 [==============================] - 0s 12ms/step - loss: 0.5820 - sparse_categorical_accuracy: 0.7600
Epoch 3/5
19/19 [==============================] - 0s 11ms/step - loss: 0.4593 - sparse_categorical_accuracy: 0.8250
Epoch 4/5
19/19 [==============================] - 0s 13ms/step - loss: 0.4244 - sparse_categorical_accuracy: 0.8183
Epoch 5/5
93/93 [==============================] - 1s 5ms/step - loss: 0.8061 - sparse_categorical_accuracy: 0.7340
No of clients in cluster 21
CLIENT NO:  15
Epoch 1/5
19/19 [==============================] - 1s 13ms/step - loss: 0.9966 - sparse_categorical_accuracy: 0.5683
Epoch 2/5
19/19 [==============================] - 0s 12ms/step - loss: 0.7138 - sparse_categorical_accuracy: 0.7317
Epoch 3/5
19/19 [==============================] - 0s 12ms/step - loss: 0.5216 - sparse_categorical_accuracy: 0.7700
Epoch 4/5
19/19 [=====

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:342: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:343: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Epoch 1/5
14/14 [==============================] - 1s 12ms/step - loss: 1.0532 - sparse_categorical_accuracy: 0.5856
Epoch 2/5
14/14 [==============================] - 0s 12ms/step - loss: 0.5972 - sparse_categorical_accuracy: 0.7365
Epoch 3/5
14/14 [==============================] - 0s 13ms/step - loss: 0.4525 - sparse_categorical_accuracy: 0.8018
Epoch 4/5
14/14 [==============================] - 0s 12ms/step - loss: 0.3794 - sparse_categorical_accuracy: 0.8266
Epoch 5/5
93/93 [==============================] - 1s 5ms/step - loss: 0.9817 - sparse_categorical_accuracy: 0.7092
No of clients in cluster 27
CLIENT NO:  6
Clnt 6 epoch 4
Epoch 1/4
17/17 [==============================] - 1s 10ms/step - loss: 1.1424 - sparse_categorical_accuracy: 0.5147
Epoch 2/4
17/17 [==============================] - 0s 11ms/step - loss: 0.6214 - sparse_categorical_accuracy: 0.7316
Epoch 3/4
17/17 [==============================] - 0s 11ms/step - loss: 0.5743 - sparse_categorical_accuracy: 0.7629
Epoch 4/

KeyboardInterrupt: ignored

In [ ]:
print(addndata)